long Quasi_Normal_Independent(long fcall, long seq, double xmean[], double std[],
long idim, double quasi[])
{
/* Input parameters:
=================
fcall
-
if fcall == 1 then it is an initialisation call,
if fcall == 0 then a continuation call
seq
-
if seq == 0 then a Faure sequence, if seq == 1 then a Niederreiter sequence,
if seq == 2 then a Sobol sequence
xmean[]
-
the means of the independent normal variates
std[]
-
the standard deviations of the independent normal variates
idim
-
the number of independent normal variates, idim must be less than 40
Output parameters:
==================
quasi[]
-
the elements quasi[0], .. quasi[idim-1] contain the independent normal variates
*/
long ierr, i, j;
double twopi, v1, v2, pi;
long ind1, ind2;
#define QUASI(I) quasi[(I)-1]
#define STD(I) std[(I)-1]
#define XMEAN(I) xmean[(I)-1]
if ((idim / 2) * 2 != idim) {
printf("Error on entry, idim is not an even number: idim = %ld\n" ,idim);
return 1;
} else if (idim > 40) {
printf("On entry, idim > 40: idim = %ld\n" ,idim);
return 1;
}
for (i = 1; i <= idim; ++i) {
if (STD(i) <= 0.0) {
printf("On entry, the standard deviation is not greater than zero:
STD(%ld) = %12.4f\n" ,i,STD(i));
return 1;
}
}
pi = 4.0*atan(1.0);
if (fcall) { /* first call for initialisation */
if (seq == 0) {
Generate_Faure_Sequence(fcall, idim, &QUASI(1));
}
else if (seq == 1) {
Generate_Niederreiter_Sequence(fcall, idim, &QUASI(1));
}
else if (seq == 2) {
Generate_Sobol_Sequence(fcall, idim, &QUASI(1));
}
} else { /* a continuation call */
if (seq == 0) {
Generate_Faure_Sequence(fcall, idim, &QUASI(1));
}
else if (seq == 1) {
Generate_ Niederreiter_Sequence(fcall, idim, &QUASI(1));
}
else if (seq == 2) {
Generate_Sobol_Sequence(fcall, idim, &QUASI(1));
}
for (i = 1; i <= idim/2; ++i) { /* generate the normal variates */
ind1 = i * 2 - 1;
ind2 = i * 2;
twopi = pi * 2.0;
v1 = sqrt(log(QUASI(ind1)) * -2.0);
v2 = twopi * QUASI(ind2);
QUASI(ind1) = XMEAN(ind1) + STD(ind1) * v1 * cos(v2);
QUASI(ind2) = XMEAN(ind2) + STD(ind2) * v1 * sin(v2);
}
}
return 0 ;
}

long Quasirandom_Normal_LogNormal_Correlated(long fcall, long seq, long lnorm,
double means[], long n, double c[], long tdc, double tol, long *irank,
double x[], double work[], long lwk) {
/* Input parameters:
=================
fcall
-
if fcall == 1 then it is an initialisation call,
if fcall == 0 then a continuation call
seq
-
if seq == 0 then a Faure sequence, if seq == 1 then a Niederreiter sequence,
if seq == 2 then a Sobol sequence
lnorm
-
if lnorm == 1 then it is a lognormal distribution,
if lnorm == 0 then a normal distribution
n
-
the number of variates, n must be less than 40
c[]
-
a matrix which contains the required covariance matrix, C
tdc
-
the second dimension of the matrix C
tol
-
the tolerance used for calculating the rank of the covariance matrix C
means[]
-
the means of the independent normal variates
std[]
-
the standard deviations of the independent normal variates
lwk
-
the size of the work array, work
Output parameters:
==================
rank
-
the computed rank of the covariance matrix C
x[]
-
the elements x[0], .. x[n-1] contain the variates
Input/Output parameters:
=========================
work
-
a work array
*/
double zero = 0.0, one = 1.0, two = 2.0;
long n1, i, j, k, kk;
double mtol, alpha;
long ptrc, ptre, ptrv, ptrw, ptrw0, ptrw1;
#define C(I,J) c[((I)-1) * tdc + ((J)-1)]
#define MEANS(I) means[(I)-1]
#define X(I) x[(I)-1]
#define WORK(I) work[(I)-1]
if (lwk < (2 + 3*n + 2*n*n + 3)) {
printf ("Error lwk is too small \n");
return 1;
}
ptre = 2;
ptrv = n+2;
ptrw = n*n + n + 2;
/* add extra 1 to allow for odd values of n */
ptrw0 = ptrw + 1 + n;
ptrw1 = ptrw0 + 1 + n;
ptrc
= ptrw1 + n + 1;
n1 = n;
if (((n/2)*2) != n) { /*
test for odd n */
n1 = n + 1;
}
if (fcall) {
/* first call for initialisation */
if (lnorm) { /* lognormal distribution */
for (i = 1; i <=n; ++i) { /* Load the modified covariance matrix into WORK */
for (j = 1; j <= n; ++j) {
WORK(ptrc+(i-1)*n+j-1) = log(one + C(i,j)/(MEANS(i)*MEANS(j)));
}
}
}
else { /* normal distribution */
for (i = 1; i <=n; ++i) {
/* Load the covariance matrix into WORK */
for (j = 1; j <= n; ++j) {
WORK(ptrc+(i-1)*n+j-1) = C(i,j);
}
}
}
/* calculate the eigenvalues and eigenvector of the matrix that
has been loaded into WORK */
calc_eigvals_eigvecs (n,&WORK(ptrc),n,&WORK(ptre),&WORK(ptrv),n);
/* The code uses NAG routine f02abc */
*irank = 0;
/*
printf ("The eigenvalues are \n");
for (j=n; j >= 1; --j) {
printf ("%12.5f \n", WORK(ptre+j-1));
}
*/
for (j=n; j >= 1; --j) { /* use the eigenvalues to calculate the rank of the matrix */
if (WORK(ptre+j-1) < tol) goto L24;
*irank = *irank + 1;
}
printf ("*irank = %ld \n",*irank);
L24:
mtol = -tol;
if (WORK(ptre) < mtol) {
printf ("Warning there is an eigenvalue less than %12.4f \n",mtol);
}
for (j=1; j <= *irank; ++j) {
kk = 1;
for (k=1; k <=n; ++k) {
if(WORK(ptrv+(k-1)*n+(j-1)) != zero) goto L28;
kk = kk + 1;
}
L28:
/* ensure that all eigenvectors have the same sign on different machines */
alpha = sqrt(WORK(ptre+j-1));
if
(WORK(ptrv+(kk-1)*n+(j-1)) < zero)
alpha = -sqrt(WORK(ptre+j-1));
for (i = 1; i <= n; ++i) {
WORK(ptrv+(j-1)+(i-1)*n)=WORK(ptrv+(j-1)+(i-1)*n)*alpha;
}
}
/*
printf ("The eigenvectors are \n");
for (j=1; j <= *irank; ++j) {
for (i = 1; i <= n; ++i) {
printf ("%10.5f ", WORK(ptrv+(j-1)+(i-1)*n));
}
printf ("\n");
}
*/
for (i = 1; i <=n; ++i)
{ /* store a vector of ones and zeros for generating the quasi-random numbers */
WORK(ptrw0+i-1) = zero;
WORK(ptrw1+i-1) = one;
for (i = n; i <= n1; ++ i) {
WORK(ptrw0+i-1) = zero;
WORK(ptrw1+i-1) = one;
}
} /* end of first
call section */
/* generate a vector of n1 random variables from a standard normal distribution,
zero mean and unit variance */
Quasi_Normal_Independent(fcall, seq, &WORK(ptrw0), &WORK(ptrw1),
n1, &WORK(ptrw));
/*
printf ("The quasi random numbers are:\n");
for (i = 1; i <= n; ++i) {
printf ("%12.4f \n", WORK(ptrw+(i-1)));
}
*/
/* Now generate variates with the specified mean and variance */
if (lnorm) { /* a lognormal distribution */
for (i = 1; i <= n; ++i) {
X(i) =
log(MEANS(i)) - WORK(ptrc+(i-1)*n+i-1)/two;
for (k = 1; k <= *irank; ++k) {
X(i)=X(i)+WORK(ptrv+(k-1)+(i-1)*n)*WORK(ptrw+k-1);
}
}
for (i = 1; i <= n; ++i) {
X(i) = exp(X(i));
}
}
else { /* a normal distribution */
for (i = 1; i <= n; ++i) {
X(i) =
MEANS(i);
for (k = 1; k <= *irank; ++k) {
X(i)=X(i)+WORK(ptrv+(k-1)+(i-1)*n)*WORK(ptrw+k-1);
}
}
}
/*
printf ("The generated variates are:\n");
for (i = 1; i <= n; ++i) {
printf (" %12.4f \n", X(i));
}
*/
return 0;
}